In [1]:
import onnx
import torch
import numpy
%config Completer.use_jedi = False

In [2]:
from torchvision.models.resnet import ResNet, Bottleneck


class ResNet152ImageClassifier(ResNet):
    def __init__(self):
        super(ResNet152ImageClassifier, self).__init__(Bottleneck, [3, 8, 36, 3])

In [3]:
batch_size = 1
x = torch.randn(batch_size, 3, 64, 64, requires_grad=True)

In [4]:
model = ResNet152ImageClassifier()

In [5]:
model.load_state_dict(torch.load("../torchserve/model-store/resnet152.pth"))

<All keys matched successfully>

In [6]:
torch.onnx.export(model,              
                  x,                        
                  "resnet152.onnx",  
                  verbose=True,
                  input_names = ['input'],  
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size', 2:'width', 3:'height'},
                                'output' : {0 : 'batch_size', 2:'width', 3:'height'}})

/home/jingch/miniconda3/envs/model-serve/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


graph(%input : Float(*, 3, *, *, strides=[12288, 4096, 64, 1], requires_grad=1, device=cpu),
      %fc.weight : Float(1000, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %1449 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %1450 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %1452 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %1453 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %1455 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %1456 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %1458 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %1459 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %1461 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %1462 : Float(256, strides=[1], req

In [7]:
model = onnx.load("resnet152.onnx")
onnx.checker.check_model(model)
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %input[FLOAT, batch_sizex3xwidthxheight]\n) initializers (\n  %fc.weight[FLOAT, 1000x2048]\n  %fc.bias[FLOAT, 1000]\n  %1449[FLOAT, 64x3x7x7]\n  %1450[FLOAT, 64]\n  %1452[FLOAT, 64x64x1x1]\n  %1453[FLOAT, 64]\n  %1455[FLOAT, 64x64x3x3]\n  %1456[FLOAT, 64]\n  %1458[FLOAT, 256x64x1x1]\n  %1459[FLOAT, 256]\n  %1461[FLOAT, 256x64x1x1]\n  %1462[FLOAT, 256]\n  %1464[FLOAT, 64x256x1x1]\n  %1465[FLOAT, 64]\n  %1467[FLOAT, 64x64x3x3]\n  %1468[FLOAT, 64]\n  %1470[FLOAT, 256x64x1x1]\n  %1471[FLOAT, 256]\n  %1473[FLOAT, 64x256x1x1]\n  %1474[FLOAT, 64]\n  %1476[FLOAT, 64x64x3x3]\n  %1477[FLOAT, 64]\n  %1479[FLOAT, 256x64x1x1]\n  %1480[FLOAT, 256]\n  %1482[FLOAT, 128x256x1x1]\n  %1483[FLOAT, 128]\n  %1485[FLOAT, 128x128x3x3]\n  %1486[FLOAT, 128]\n  %1488[FLOAT, 512x128x1x1]\n  %1489[FLOAT, 512]\n  %1491[FLOAT, 512x256x1x1]\n  %1492[FLOAT, 512]\n  %1494[FLOAT, 128x512x1x1]\n  %1495[FLOAT, 128]\n  %1497[FLOAT, 128x128x3x3]\n  %1498[FLOAT, 128]\n  %1500[FLOAT, 512x128x1x1]\

In [8]:
import onnxruntime as ort
import numpy as np

In [9]:
ort_session = ort.InferenceSession('resnet152.onnx')

In [10]:
import matplotlib.pyplot as plt
import time

In [11]:
imgs = []

for i in range(981):
    file = f"../images/test_{i}.JPEG"
    img = plt.imread(file)
    imgs.append(img.astype(np.float32).reshape(3, 64, 64))
imgs = np.array(imgs)

In [ ]:
ort_session.run(None, {'input': imgs[0:1]})

In [17]:
ort_session.run(None, {'input': np.random.rand(1, 3, 1120, 1120).astype(np.float32)})

[array([[-2.63646984e+00,  8.83059800e-01,  1.18525195e+00,
          7.59211600e-01,  9.47800517e-01, -7.32815504e-01,
          6.05881214e-05, -1.06815577e+00, -1.83111131e+00,
         -2.78042269e+00,  1.40992594e+00,  2.16158891e+00,
          2.97277617e+00,  1.59423375e+00,  2.35681915e+00,
          1.30566716e+00,  1.37987554e+00, -1.39131701e+00,
          1.64184690e+00,  2.06483412e+00, -5.01269102e-01,
          2.71896386e+00,  7.71595955e-01,  3.97731453e-01,
          1.44462514e+00, -2.24554753e+00, -1.16223931e-01,
          9.25781071e-01, -1.58936751e+00, -3.41705155e+00,
         -2.98019099e+00,  4.94110048e-01, -2.51070976e+00,
         -4.73824143e-01,  6.83626711e-01, -3.64278364e+00,
         -1.29081714e+00, -2.45536113e+00,  1.60703862e+00,
         -2.67561340e+00,  1.30866063e+00, -1.16371334e+00,
          2.31182307e-01, -6.72121197e-02,  6.26669079e-03,
         -2.44813919e-01,  8.22794914e-01, -3.21876764e-01,
         -2.69398475e+00, -2.47549558e+0

In [13]:
t1 = time.time()
for i in range(981):
    print(i)
    output = ort_session.run(None, {'input': imgs[i:i+1]})
print(time.time() - t1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27